# Understanding Trends in Company Valuation with NLP - Part 3: NLP Company Earnings Analysis using SageMaker Asynchronous Inference and AWS Lambda

## Introduction

### Orchestrating company earnings trend analysis, using SEC filings, news sentiment with the Hugging Face transformers, and Amazon SageMaker Pipelines

In this notebook, we demonstrate how to summarize and derive sentiments out of Security and Exchange Commission reports filed by a publicly traded organization. We will derive the overall market sentiments about the said organization through financial news articles within the same financial period to present a fair view of the organization vs. market sentiments and outlook about the company's overall valuation and performance. In addition to this we will also identify the most popular keywords and entities within the news articles about that organization.

In order to achieve the above we will be using multiple SageMaker Hugging Face based NLP transformers for the downstream NLP tasks of Summarization (e.g., of the news and SEC MDNA sections) and Sentiment Analysis (of the resulting summaries).

---

### Using SageMaker Pipelines

Amazon SageMaker Pipelines is the first purpose-built, easy-to-use continuous integration and continuous delivery (CI/CD) service for machine learning (ML). With SageMaker Pipelines, you can create, automate, and manage end-to-end ML workflows at scale. 

Orchestrating workflows across each step of the machine learning process (e.g. exploring and preparing data, experimenting with different algorithms and parameters, training and tuning models, and deploying models to production) can take months of coding.

Since it is purpose-built for machine learning, SageMaker Pipelines helps you automate different steps of the ML workflow, including data loading, data transformation, training and tuning, and deployment. With SageMaker Pipelines, you can build dozens of ML models a week, manage massive volumes of data, thousands of training experiments, and hundreds of different model versions. You can share and re-use workflows to recreate or optimize models, helping you scale ML throughout your organization.

---

### Understanding trends in company valuation (or similar) with NLP

**Natural language processing (NLP)** is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves. (Source: [Wikipedia](https://en.wikipedia.org/wiki/Natural_language_processing))

We are going to demonstrate how to summarize and derive sentiments out of Security and Exchange Commission reports filed by a publicly traded organization. We are also going to derive the overall market sentiments about the said organization through financial news articles within the same financial period to present a fair view of the organization vs. market sentiments and outlook about the company's overall valuation and performance. In addition to this we will also identify the most popular keywords and entities within the news articles about that organization.

In order to achieve the above we will be using multiple SageMaker Hugging Face based NLP transformers with summarization and sentiment analysis downstream tasks.

* <b> Summarization of financial text from SEC reports and news articles </b> will be done via [Pegasus for Financial Summarization model](https://huggingface.co/human-centered-summarization/financial-summarization-pegasus) based on the paper [Towards Human-Centered Summarization: A Case Study on Financial News](https://aclanthology.org/2021.hcinlp-1.4/). 
* Sentiment analysis on summarized SEC financial report and news articles will be done via pre-trained NLP model to analyze sentiment of financial text called [FinBERT](https://huggingface.co/ProsusAI/finbert). Paper: [ FinBERT: Financial Sentiment Analysis with Pre-trained Language Models](https://arxiv.org/abs/1908.10063)

---

### SEC Dataset

The starting point for a vast amount of financial NLP is text in SEC filings. The SEC requires companies to report different types of information related to various events involving companies. The full list of SEC forms is here: https://www.sec.gov/forms.

SEC filings are widely used by financial services companies as a source of information about companies in order to make trading, lending, investment, and risk management decisions. Because these filings are required by regulation, they are of high quality and veracity. They contain forward-looking information that helps with forecasts and are written with a view to the future, required by regulation. In addition, in recent times, the value of historical time-series data has degraded, since economies have been structurally transformed by trade wars, pandemics, and political upheavals. Therefore, text as a source of forward-looking information has been increasing in relevance. 

#### Obtain the dataset using the SageMaker JumpStart Industry Python SDK

Downloading SEC filings is done from the SEC's Electronic Data Gathering, Analysis, and Retrieval (EDGAR) website, which provides open data access. EDGAR is the primary system under the U.S. Securities And Exchange Commission (SEC) for companies and others submitting documents under the Securities Act of 1933, the Securities Exchange Act of 1934, the Trust Indenture Act of 1939, and the Investment Company Act of 1940. EDGAR contains millions of company and individual filings. The system processes about 3,000 filings per day, serves up 3,000 terabytes of data to the public annually, and accommodates 40,000 new filers per year on average.

There are several ways to download the data, and some open source packages available to extract the text from these filings. However, these require extensive programming and are not always easy-to-use. We provide a simple one-API call that will create a dataset in a few lines of code, for any period of time and for numerous tickers.

We have wrapped the extraction functionality into a SageMaker processing container and provide this notebook to enable users to download a dataset of filings with metadata such as dates and parsed plain text that can then be used for machine learning using other SageMaker tools. This is included in the [SageMaker Industry Jumpstart Industry](https://aws.amazon.com/blogs/machine-learning/use-pre-trained-financial-language-models-for-transfer-learning-in-amazon-sagemaker-jumpstart/) library for financial language models. Users only need to specify a date range and a list of ticker symbols, and the library will take care of the rest.

As of now, the solution supports extracting a popular subset of SEC forms in plain text (excluding tables): 10-K, 10-Q, 8-K, 497, 497K, S-3ASR, and N-1A. For each of these, we provide examples throughout this notebook and a brief description of each form. For the 10-K and 10-Q forms, filed every year or quarter, we also extract the Management Discussion and Analysis (MDNA) section, which is the primary forward-looking section in the filing. This is the section that has been most widely used in financial text analysis. Therefore, we provide this section automatically in a separate column of the dataframe alongside the full text of the filing.

The extracted dataframe is written to S3 storage and to the local notebook instance. 

---

### News articles related to the stock symbol -- dataset

We will use the MIT Licensed [NewsCatcher API](https://docs.newscatcherapi.com/) to grab top 4-5 articles about the specific organization using filters, however other sources such as Social media feeds, RSS Feeds can also be used.


The first step in the pipeline is to fetch the SEC report from the EDGAR database using the [SageMaker Industry Jumpstart Industry](https://aws.amazon.com/blogs/machine-learning/use-pre-trained-financial-language-models-for-transfer-learning-in-amazon-sagemaker-jumpstart/) library for Financial language models. This library provides us an easy to use functionality to obtain either one or multiple SEC reports for one or more Ticker symbols or CIKs. The ticker or CIK number will be passed to the SageMaker Pipeline using Pipeline parameter `inference_ticker_cik`. For demo purposes of this Pipeline we will focus on a single Ticker/CIK number at a time and the MDNA section of the 10-K form. The first processing will extract the MDNA from the 10-K form for a company and will also gather few news articles related to the company from the NewsCatcher API. This data will ultimately be used for summarization and then finally sentiment analysis.

---

## MLOps for NLP using SageMaker Pipelines

We will set up the following SageMaker Pipeline. The Pipleline has two flows depending on what the value for `model_register_deploy` Pipeline parameter is set to. If the value is set to `Y` we want the pipeline to register the model and deploy the latest version of the model from the model registry to the SageMaker endpoint. If the value is set to `N` then we simply want to run inferences using the [FinBERT](https://huggingface.co/ProsusAI/finbert) and the [Pegasus](https://huggingface.co/transformers/model_doc/pegasus.html) models using the Ticker symbol (or CIK number) that is passed to the pipeline using the `inference_ticker_cik` Pipeline parameter.

<img src="./images/pipeline_lambda_execution_graph.png" alt="Pipeline" style="width: 800px;"/>

<div class="alert alert-block alert-warning">
    <b>Note:</b> You must execute the <code>01_nlp_setup_company_earnings_analysis.ipynb</code> notebook before you can set up the SageMaker Pipeline. In that notebook we create a custom Docker image and register it in Amazon Elastic Container Registry (Amazon ECR) for the pipeline to use. The image contains of all the required dependencies.
</div>

---

## Set Up SageMaker Project

<a id='setup-project'></a>


### Install and import packages

In [ ]:
# Install updated version of SageMaker
# !pip install -q sagemaker==2.49
!pip install sagemaker --upgrade
# !pip install transformers
# !pip install typing
# !pip install sentencepiece
# !pip install fiscalyear

In [ ]:
# Install SageMaker JumpStart Industry
!pip install smjsindustry

<div class="alert alert-block alert-warning">
NOTE: After installing an updated version of SageMaker and PyTorch, save the notebook and then restart your kernel.
</div>

In [ ]:
import boto3
import botocore
import pandas as pd
import sagemaker
import time

print(f"SageMaker version: {sagemaker.__version__}")

from sagemaker.huggingface import HuggingFace
from sagemaker.huggingface import HuggingFaceModel

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.parameters import ParameterInteger, ParameterString
from sagemaker.sklearn.processing import ScriptProcessor
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

### Define parameters that you'll use throughout the notebook

In [ ]:
s3 = boto3.resource("s3")
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sagemaker_role = role
default_bucket = sagemaker_session.default_bucket()
prefix = "nlp-e2e-mlops"
s3_client = boto3.client("s3", region_name=region)
sagemaker_boto_client = boto3.client("sagemaker", region_name=region)


# deploy_model_instance_type = "ml.m4.8xlarge"
deploy_model_instance_type = "ml.m4.xlarge"
inference_instances = [
    "ml.t2.medium",
    "ml.m5.xlarge",
    "ml.m5.2xlarge",
    "ml.m5.4xlarge",
    "ml.m5.12xlarge",
]
transform_instances = ["ml.m5.xlarge"]
PROCESSING_INSTANCE = "ml.m4.4xlarge"
ticker = "AMZN"

%store -r

In [ ]:
print(f"s3://{default_bucket}/{prefix}/code/model_deploy.py")
print(f"SageMaker Role: {role}")

### Define parameters to parametrize Pipeline Execution

Using SageMaker Pipelines, we can define the steps to be included in a pipeline but then use parameters to modify that pipeline when we go to execute the pipeline, without having to modify the pipeline definition. We'll provide some default parameter values that can be overridden on pipeline execution.

In [ ]:
# Define some default parameters:

# specify default number of instances for processing step
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

# specify default instance type for processing step
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value=PROCESSING_INSTANCE
)

# specify location of inference data for data processing step
inference_input_data = ParameterString(
    name="InferenceData",
    default_value=f"s3://{default_bucket}/{prefix}/nlp-pipeline/inf-data",
)

# Specify the Ticker CIK for the pipeline
inference_ticker_cik = ParameterString(
    name="InferenceTickerCik",
    default_value=ticker,
)

# specify default method for model approval
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

# specify if new model needs to be registered and deployed
model_register_deploy = ParameterString(name="ModelRegisterDeploy", default_value="Y")

In [ ]:
%store
# These are the stored variables, the container is created in the
# previous notebook 01_script-processor-custom-container.ipynb

In [ ]:
%pylab inline
%store -r

---

<a id='analyze-sec'></a>

## Preparing SEC dataset

Before we dive right into setting up the pipeline, let's take a look at how the SageMaker Jumpstart Industry SDK for Financial language model helps obtain the dataset from SEC forms and what are the features available for us to use. 

**Note:** The code cells in this section are completely optional and for information purposes only; we will use the SageMaker JumpStart Industry SDK directly in the pipeline. 

Let's install the required dependencies first.

### Install the SageMaker JumpStart Industry SDK

The functionality is delivered through a client-side SDK. The first step requires pip installing a Python package that interacts with a SageMaker processing container. The retrieval, parsing, transforming, and scoring of text is a complex process and uses different algorithms and packages. In order to make this seamless and stable for the user, the functionality is packaged into a SageMaker container. This lifts the load of installation and maintenance of the workflow, reducing the user effort down to a pip install followed by a single API call. 

As an example, we will try to pull AMZN ticker 10k/10q filings from EDGAR and write the data as CSV to S3. Below is the single block of code that contains the API call. The options are all self-explanatory. 

In [ ]:
# from smfinance import SECDataSetConfig, DataLoader
from smjsindustry.finance import DataLoader
from smjsindustry.finance.processor_config import EDGARDataSetConfig

The extracted reports will be saved to an S3 bucket for us to review. This code will also be used in the Pipeline to fetch the report for the Ticker or CIK number passed to the SageMaker Pipeline. Executing the following code cell will run a processing job which will fetch the SEC reports from the EDGAR database.

<a id='use-smjsindustry'></a>

### Obtain SEC data using the SageMaker JumpStart Industry SDK

In [ ]:
%%time

dataset_config = EDGARDataSetConfig(
    tickers_or_ciks=["amzn", "goog", "27904", "FB"],  # list of stock tickers or CIKs
    form_types=["10-K", "10-Q"],  # list of SEC form types
    filing_date_start="2019-01-01",  # starting filing date
    filing_date_end="2020-12-31",  # ending filing date
    email_as_user_agent="test-user@test.com",
)  # user agent email

data_loader = DataLoader(
    role=sagemaker.get_execution_role(),  # loading job execution role
    instance_count=1,  # instances number, limit varies with instance type
    instance_type="ml.c5.2xlarge",  # instance type
    volume_size_in_gb=30,  # size in GB of the EBS volume to use
    volume_kms_key=None,  # KMS key for the processing volume
    output_kms_key=None,  # KMS key ID for processing job outputs
    max_runtime_in_seconds=None,  # timeout in seconds. Default is 24 hours.
    sagemaker_session=sagemaker.Session(),  # session object
    tags=None,
)  # a list of key-value pairs

data_loader.load(
    dataset_config,
    "s3://{}/{}".format(
        default_bucket, "sample-sec-data"
    ),  # output s3 prefix (both bucket and folder names are required)
    "dataset_10k_10q.csv",  # output file name
    wait=True,
    logs=True,
)

#### Output

The output of the `data_loader` processing job is a `CSV` file. We see the filings for different quarters. 

The filing date comes within a month of the end date of the reporting period. Both these dates are collected and displayed in the dataframe. The column `text` contains the full text of the report, but the tables are not extracted. The values in the tables in the filings are balance-sheet and income-statement data (numeric/tabular) and are easily available elsewhere as they are reported in numeric databases. The last column of the dataframe comprises the Management Discussion & Analysis section, the column is named `mdna`, which is the primary forward-looking section in the filing. This is the section that has been most widely used in financial text analysis. Therefore, we will use the `mdna` text to derive the sentiment of the overall filing in this example. 

In [ ]:
print(f"{default_bucket}/{prefix}/")
s3_client.download_file(
    default_bucket,
    "{}/{}".format(f"sample-sec-data", f"dataset_10k_10q.csv"),
    f"./data/dataset_10k_10q.csv",
)

In [ ]:
data_frame_10k_10q = pd.read_csv(f"./data/dataset_10k_10q.csv")
data_frame_10k_10q

---

## Set Up Your MLOps NLP Pipeline with SageMaker Pipelines

<a id='pipe-pre-deploy'></a>

### Step 1: Data pre-processing - extract SEC data and news about the company

#### Define a processing step to prepare SEC data for inference

We will define a processing step to extract 10K and 10Q forms for a specific Organization either using the company [Stock Ticker](https://www.investopedia.com/ask/answers/12/what-is-a-stock-ticker.asp) Symbol or [CIK (Central Index Key)](https://www.sec.gov/edgar/searchedgar/cik.htm) used to lookup reports in SEC's EDGAR System. You can find the company Stock Ticker Symbol to CIK Number mapping [here](https://www.sec.gov/include/ticker.txt). This step will also collect news article snippets related to the company using the NewsCatcher API.

#### **<span style="color:lightgreen">Important</span>**:

It is recommended to use CIKs as the input. The tickers will be internally converted to CIKs according to the [mapping file](https://www.sec.gov/include/ticker.txt).  
One ticker may map to multiple CIKs, but we only support the latest ticker to CIK mapping. Please provide the old CIKs in the input when you want historical filings. Also note that even though the Client side SDK allows you to download multiple SEC reports for multiple CIKs at a time, we will set up our data preprocessing step to grab exactly 1 SEC Report for 1 CIK (Company/Organization). 


In [ ]:
"""
we used store magic in the previous note book script-processor-custom-container.ipynb 
to instantiate the container in the region of choice 
"""
CONTAINER_IMAGE_URI

In [ ]:
loader_instance_type = "ml.c5.2xlarge"
create_dataset_processor = ScriptProcessor(
    command=["python3"],
    image_uri=CONTAINER_IMAGE_URI,
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

Create a processing step to process the SEC data for inference:

In [ ]:
create_dataset_script_uri = f"s3://{default_bucket}/{prefix}/code/data-processing.py"
s3_client.upload_file(
    Filename="./scripts/data-processing.py",
    Bucket=default_bucket,
    Key=f"{prefix}/code/data-processing.py",
)

create_dataset_step = ProcessingStep(
    name="nlp-e2e-FinBertCreateDataset",
    processor=create_dataset_processor,
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="report_data",
            source="/opt/ml/processing/output/10k10q",
            destination=f"{inference_input_data}/10k10q",
        ),
        sagemaker.processing.ProcessingOutput(
            output_name="article_data",
            source="/opt/ml/processing/output/articles",
            destination=f"{inference_input_data}/articles",
        ),
    ],
    job_arguments=[
        "--ticker-cik",
        inference_ticker_cik,
        "--instance-type",
        loader_instance_type,
        "--region",
        region,
        "--bucket",
        default_bucket,
        "--prefix",
        prefix,
        "--role",
        role,
    ],
    code=create_dataset_script_uri,
)

<a id='pipe-pre-deploy'></a>

### Step 2: Create models for summarization and sentiment analysis

In [ ]:
sentiment_model_name = "nlp-e2e-FinbertModel"
summarization_model_name = "nlp-e2e-PegasusModel"

#### Create the `finBert` model for Sentiment Analysis

In [ ]:
# Download pre-trained model using HuggingFaceModel class
from sagemaker.huggingface import HuggingFaceModel

hub = {"HF_MODEL_ID": "ProsusAI/finbert", "HF_TASK": "text-classification"}

# create Hugging Face Model Class (documentation here: https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model)
sentiment_huggingface_model = HuggingFaceModel(
    name=sentiment_model_name,
    transformers_version="4.6.1",
    pytorch_version="1.7.1",
    py_version="py36",
    env=hub,
    role=role,
    sagemaker_session=sagemaker_session,
)

inputs = sagemaker.inputs.CreateModelInput(instance_type="ml.m4.xlarge")

# This may not be needed since we are going to use Model registry
# create_sentiment_model_step = CreateModelStep(
#     name="nlp-e2e-FinBertCreateModel",
#     model=sentiment_huggingface_model,
#     inputs=inputs
# )

#### Create the Pegasus summarization model

In [ ]:
hub = {
    "HF_MODEL_ID": "human-centered-summarization/financial-summarization-pegasus",
    "HF_TASK": "summarization",
}

# create Hugging Face Model Class (documentation here: https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model)
summary_huggingface_model = HuggingFaceModel(
    name=summarization_model_name,
    transformers_version="4.6.1",
    pytorch_version="1.7.1",
    py_version="py36",
    env=hub,
    role=role,
    sagemaker_session=sagemaker_session,
)

# This may not be needed since we are going to use Model registry
# create_summary_model_step = CreateModelStep(
#     name="nlp-e2e-PegasusCreateModel",
#     model=summary_huggingface_model,
#     inputs=inputs,
# )

<a id='model-registry'></a>

### Step 3: Register model

Use HuggingFace register method to register Hugging Face Model for deployment. Set up step as a custom processing step

In [ ]:
sentiment_model_package_group_name = "HuggingFaceSECSentimentModelPackageGroup"
summary_model_package_group_name = "HuggingFaceSECSummaryModelPackageGroup"
model_approval_status = "Approved"

register_sentiment_model_step = RegisterModel(
    name="nlp-e2e-FinBertRegisterModel",
    model=sentiment_huggingface_model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=inference_instances,
    transform_instances=transform_instances,
    model_package_group_name=sentiment_model_package_group_name,
    approval_status=model_approval_status,
    #     depends_on=['nlp-e2e-FinBertCreateModel']
)

register_summary_model_step = RegisterModel(
    name="nlp-e2e-PegasusRegisterModel",
    model=summary_huggingface_model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=inference_instances,
    transform_instances=transform_instances,
    model_package_group_name=summary_model_package_group_name,
    approval_status=model_approval_status,
    #     depends_on=['nlp-e2e-PegasusCreateModel']
)

<a id='deploy'></a>

### Step 4: Deploy model using Lambda step


We deploy the FinBert and Pegasus models from the model registry. 

**NOTE:** The models in the model registry are the pre-trained version from HuggingFace Model Hub. Each of the deployment step will attempt to deploy a SageMaker Endpoint with the model and will write a property file upon successful completion. The Pipeline will make use of these property files to decide whether to execute the subsequent summarization and sentiment analysis inference steps.

We can also deploy a HuggingFace model similar to how we deployed the FinBert sentiment model above but using SageMaker Pipelines `Lambda` step. The SageMaker SDK provides a Lambda helper class that can be used to create a Lambda function. This function is provided to the Lambda step for invocation via the pipeline. Alternatively, a predefined Lambda function can be provided to the Lambda step. 

The SageMaker Execution Role requires the policy `AmazonSageMakerPipelinesIntegrations` to create the Lambda function, and the Lambda function needs a role with policies allowing creation of a SageMaker endpoint. The AWS Lambda execution role must have `AWSLambdaBasicExecutionRole` and `AmazonSageMakerFullAccess` managed Policies. The IAM role must also have the trust policy shown below. For More information on how to create a AWS Lambda execution role, refer to [documentation](https://docs.aws.amazon.com/lambda/latest/dg/lambda-intro-execution-role.html).

<code style="color:#00E676;font-size:12px">
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
</code>

<div class="alert alert-warning"> <h4 color="red"><strong>💡 NOTE </strong></h4>
The IAM Policies described above can be overly permissive. Please practice caution in setting up IAM Roles with them. For more information about fine-grained permissions for the sagemaker-studio-image-build tool, see <a href="https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/">Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks</a>. For best practices on SageMaker security, IAM roles, and policies, see <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/security_iam_id-based-policy-examples.html#security_iam_service-with-iam-policy-best-practices">Policy Best Practices</a> in the Amazon SageMaker Developer Guide.
</div>

In the following cell, specify the name of AWS Lambda execution role. You may replace it with the role name you specified.

In [ ]:
lambda_role_name = "<sagemaker-pipelines-lambda-step-role>"
lambda_role = boto3.client("iam").get_role(RoleName=lambda_role_name)["Role"]["Arn"]
lambda_role

We will create a few variables to be used for setting up the LambdaStep

In [ ]:
deploy_model_instance_type = "ml.m4.4xlarge"
deploy_model_instance_count = 1

sentiment_endpoint_name = "nlp-e2e-FinBertModel-endpoint"
summarization_endpoint_name = "nlp-e2e-PegasusModel-endpoint"

The Lambda function code is contained in a a script called `lambda_deployer.py` in the `/scripts` directory.

Next, we define the Lambda step.  After defining some object names, we use the Lambda helper class to create the actual Lambda function, then pass it to the Lambda step.

In [ ]:
# Use the current time to define unique names for the resources created
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
function_name = "hfnlp-demo-lambda-deploy-step-" + current_time

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="./lambda/lambda_deployer.py",
    handler="lambda_deployer.lambda_handler",
    timeout=600,
    memory_size=10240,
)

# The dictionary retured by the Lambda function is captured by LambdaOutput, each key in the dictionary corresponds to a
# LambdaOutput

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(
    output_name="endpoint_created", output_type=LambdaOutputTypeEnum.String
)

sentiment_deploy_lambda_step = LambdaStep(
    name="LambdaStepFinbertDeploy",
    lambda_func=func,
    inputs={
        "endpoint_name": sentiment_endpoint_name,
        "deploy_model_instance_type": deploy_model_instance_type,
        "deploy_model_instance_count": deploy_model_instance_count,
        "model_package_group_name": sentiment_model_package_group_name,
        "role": role,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
    depends_on=["nlp-e2e-FinBertRegisterModel"],
)

# The inputs provided to the Lambda function can be retrieved via the `event` object within the `lambda_handler` function
# in the Lambda
summary_deploy_lambda_step = LambdaStep(
    name="LambdaStepPegasusDeploy",
    lambda_func=func,
    inputs={
        "endpoint_name": summarization_endpoint_name,
        "deploy_model_instance_type": deploy_model_instance_type,
        "deploy_model_instance_count": deploy_model_instance_count,
        "model_package_group_name": summary_model_package_group_name,
        "role": role,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
    depends_on=["nlp-e2e-PegasusRegisterModel"],
)

#### Create pipeline conditions to check if the Endpoint deployments were successful

We will define a condition that checks to see if our model deployment was successful based on the response returned by the Lambda function in the Lambda deployment steps of both the FinBert and Pegasus Models. If both the conditions evaluates to `True` then we will run or subsequent inferences for Summarization and Sentiment analysis. 

At this time we will simply define the conditions, and later when we are ready with the inference steps we will setup the `ConditionStep` using `sentiment_condition_eq` and `summary_condition_eq` conditions.

In [ ]:
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

sentiment_condition_eq = ConditionEquals(
    left=sentiment_deploy_lambda_step.properties.Outputs["endpoint_created"],
    right="Y",  # the right value of the evaluation expression, i.e. endpoint_created='Y'
)

summary_condition_eq = ConditionEquals(
    left=summary_deploy_lambda_step.properties.Outputs["endpoint_created"],
    right="Y",  # the right value of the evaluation expression, i.e. endpoint_created='Y'
)

### Step 5: Summarize SEC report step

This step is to make use of the Pegasus Summarizer model endpoint to summarize the MDNA text from the SEC report. Because the MDNA text is usually large, we want to derive a short summary of the overall text to be able to determine the overall sentiment.

In [ ]:
summarize_processor = ScriptProcessor(
    command=["python3"],
    image_uri=CONTAINER_IMAGE_URI,
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

In [ ]:
summarize_script_uri = f"s3://{default_bucket}/{prefix}/code/summarize.py"
s3_client.upload_file(
    Filename="./scripts/summarize.py", Bucket=default_bucket, Key=f"{prefix}/code/summarize.py"
)

summarize_step_1 = ProcessingStep(
    name="nlp-e2e-PegasusSummarizer_1",
    processor=summarize_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="summary_data",
            source=f"{inference_input_data}/10k10q",
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="summarized_data",
            source="/opt/ml/processing/output",
            destination=f"{inference_input_data}/10k10q/summary",
        )
    ],
    job_arguments=[
        "--ticker-cik",
        inference_ticker_cik,
        "--region",
        region,
        "--endpoint-name",
        summarization_endpoint_name,
    ],
    code=summarize_script_uri,
)

summarize_step_2 = ProcessingStep(
    name="nlp-e2e-PegasusSummarizer_2",
    processor=summarize_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="summary_data",
            source=f"{inference_input_data}/10k10q",
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="summarized_data",
            source="/opt/ml/processing/output",
            destination=f"{inference_input_data}/10k10q/summary",
        )
    ],
    job_arguments=[
        "--ticker-cik",
        inference_ticker_cik,
        "--region",
        region,
        "--endpoint-name",
        summarization_endpoint_name,
    ],
    code=summarize_script_uri,
)

### Step 6: Sentiment inference step - SEC summary and news articles

This step uses the MDNA summary (determined by the previous step) and the news articles to find out the sentiment of the company's financial and what the Market trends are indicating. This would help us understand the overall position of the company's financial outlook and current position without leaning solely on the company's forward-looking statements and bring objective market opinions into the picture.

In [ ]:
sentiment_processor = ScriptProcessor(
    command=["python3"],
    image_uri=CONTAINER_IMAGE_URI,
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
)

In [ ]:
sentiment_script_uri = f"s3://{default_bucket}/{prefix}/code/sentiment.py"
s3_client.upload_file(
    Filename="./scripts/sentiment.py", Bucket=default_bucket, Key=f"{prefix}/code/sentiment.py"
)

sentiment_step_1 = ProcessingStep(
    name="nlp-e2e-FinBertSentiment_1",
    processor=summarize_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="sec_summary",
            source=f"{inference_input_data}/10k10q/summary",
            destination="/opt/ml/processing/input/10k10q",
        ),
        sagemaker.processing.ProcessingInput(
            input_name="articles",
            source=f"{inference_input_data}/articles",
            destination="/opt/ml/processing/input/articles",
        ),
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="sentiment_data",
            source="/opt/ml/processing/output",
            destination=f"{inference_input_data}/sentiment",
        )
    ],
    job_arguments=[
        "--ticker-cik",
        inference_ticker_cik,
        "--region",
        region,
        "--endpoint-name",
        sentiment_endpoint_name,
    ],
    code=sentiment_script_uri,
    depends_on=["nlp-e2e-PegasusSummarizer_1"],
)

sentiment_step_2 = ProcessingStep(
    name="nlp-e2e-FinBertSentiment_2",
    processor=summarize_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            input_name="sec_summary",
            source=f"{inference_input_data}/10k10q/summary",
            destination="/opt/ml/processing/input/10k10q",
        ),
        sagemaker.processing.ProcessingInput(
            input_name="articles",
            source=f"{inference_input_data}/articles",
            destination="/opt/ml/processing/input/articles",
        ),
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="sentiment_data",
            source="/opt/ml/processing/output",
            destination=f"{inference_input_data}/sentiment",
        )
    ],
    job_arguments=[
        "--ticker-cik",
        inference_ticker_cik,
        "--region",
        region,
        "--endpoint-name",
        sentiment_endpoint_name,
    ],
    code=sentiment_script_uri,
    depends_on=["nlp-e2e-PegasusSummarizer_2"],
)

### Setup Condition Steps

As explained earlier, this is a top level condition step. This step will determine based on the value of the pipeline parameter `model_register_deploy` on whether we want to register and deploy a new version of the models and then run inference, or to simply run inference using the existing endpoints.

In [ ]:
from sagemaker.workflow.conditions import ConditionEquals
from sagemaker.workflow.condition_step import ConditionStep

condition_eq = ConditionEquals(left=model_register_deploy, right="Y")

In [ ]:
# Define the condition step
condition_step = ConditionStep(
    name="nlp-e2e-FinBertConditionCheck",
    conditions=[condition_eq],  # the parameter is Y
    if_steps=[
        #               create_sentiment_model_step, #CreateModel Step
        register_sentiment_model_step,
        sentiment_deploy_lambda_step,
        #               create_summary_model_step,  #CreateModel Step
        register_summary_model_step,
        summary_deploy_lambda_step,
    ],  # if the condition evaluates to true then create model, register, and deploy
    else_steps=[summarize_step_1],
    depends_on=["nlp-e2e-FinBertCreateDataset"],
)

We will also setup the `ConditionStep` using the two conditions we set up earlier - `sentiment_condition_eq` and `summary_condition_eq`. This condition will make sure that the inference steps are only executed if the endpoint deployment/update is successful (done by the LambdaStep)

In [ ]:
deploy_condition_step = ConditionStep(
    name="nlp-e2e-EndpointDeployConditionCheck",
    conditions=[
        sentiment_condition_eq,
        summary_condition_eq,
    ],  # the equal to conditions defined above
    if_steps=[
        summarize_step_2
    ],  # if the condition evaluates to true then run the summarization step
    else_steps=[],  # there are no else steps so we will keep it empty
    depends_on=[
        "LambdaStepFinbertDeploy",
        "LambdaStepPegasusDeploy",
    ],  # dependencies on both Finbert and Pegasus Deployment steps
)

#### Combine Pipeline steps and run

In [ ]:
pipeline_name = "Lambda-Company-Earnings-Sentiment-Async-Pipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type,
        processing_instance_count,
        model_register_deploy,
        inference_ticker_cik,
        inference_input_data,
    ],
    steps=[
        create_dataset_step,
        condition_step,
        deploy_condition_step,
        sentiment_step_1,
        sentiment_step_2,
    ],
)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
%%time
start_response = pipeline.start()
start_response.wait(delay=60, max_attempts=200)
start_response.describe()

The following image shows a successful execution of the NLP end-to-end Pipeline.



<img src="./images/pipeline_lambda_execution_graph.png" alt="Successful Pipeline Execution" style="width: 800px;"/>

---

## View Evaluation Results

Once the pipeline execution completes, we can download the evaluation data from S3 and view it.

In [ ]:
s3_client.download_file(
    default_bucket,
    f"{prefix}/nlp-pipeline/inf-data/sentiment/{ticker}_sentiment_result.csv",
    f"./data/{ticker}_sentiment_result.csv",
)
sentiment_df = pd.read_csv(f"./data/{ticker}_sentiment_result.csv")
sentiment_df

---

## Clean up

Delete the SageMaker Pipeline and the SageMaker Endpoints created by the pipeline.

In [ ]:
def clean_up_resources():
    pipeline.delete()
    sagemaker_boto_client.delete_endpoint(EndpointName=sentiment_endpoint_name)
    sagemaker_boto_client.delete_endpoint(EndpointName=summarization_endpoint_name)